In [9]:
import collections
import json
import pickle
import pyLDAvis
import random
import scipy.sparse
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zstandard as zstd

from pyspark import SparkContext
from pyspark.sql import SparkSession

# Select select the data

### Selection criteria
- Not used to train the topic modelling
- Video with more than 10'000 views

In [10]:
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''


    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode(errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]

In [11]:
# Load set of videos to consider
with open('/dlabdata1/youtube_large/olam/data/view10000_sub10000/idx_vid_to_consider.pickle', 'rb') as f:
    idx_vid_to_consider = pickle.load(f)
f.close()

In [14]:
# Load channels
df_channelcrawler = pd.read_csv('/dlabdata1/youtube_large/channelcrawler.csv')

df_channelcrawler['channel_id'] = df_channelcrawler['link'].apply(lambda x: x.replace('http://www.youtube.com/channel/', ''))

# filter the channels
df_channelcrawler_100000sub = df_channelcrawler[df_channelcrawler['subscribers'] >= 100000]
set_channelcrawler_100000sub = set(df_channelcrawler_100000sub['channel_id'])

In [13]:
reader = Zreader("/dlabdata1/youtube_large/yt_metadata_all.jsonl.zst", chunk_size=2**28)

idx = 0
array_relevant_infos = []

for line in reader.readlines():
    ###start_iter = time.time()
    idx += 1
    
    if idx % 1000000 == 0:
        print('Progress: ' + str(int(idx/1000000)) + '/85')
        
    if idx in idx_vid_to_consider:
        
        # line is a str dict, video is the dict corresponding to the str dict
        video = json.loads(line)
        
        array_vid_relevant_infos = [video['channel_id']]
        array_vid_relevant_infos.append(video['view_count'])
        array_vid_relevant_infos.append(video['upload_date'][:4])
        array_vid_relevant_infos.append(video['categories'])
        
        array_relevant_infos.append(array_vid_relevant_infos)
        
    

Progress: 1/85
Progress: 2/85
Progress: 3/85
Progress: 4/85
Progress: 5/85
Progress: 6/85
Progress: 7/85
Progress: 8/85
Progress: 9/85
Progress: 10/85
Progress: 11/85
Progress: 12/85
Progress: 13/85
Progress: 14/85
Progress: 15/85
Progress: 16/85
Progress: 17/85
Progress: 18/85
Progress: 19/85
Progress: 20/85
Progress: 21/85
Progress: 22/85
Progress: 23/85
Progress: 24/85
Progress: 25/85
Progress: 26/85
Progress: 33/85
Progress: 34/85
Progress: 35/85
Progress: 36/85
Progress: 37/85
Progress: 38/85
Progress: 39/85
Progress: 40/85
Progress: 41/85
Progress: 42/85
Progress: 43/85
Progress: 44/85
Progress: 45/85
Progress: 46/85
Progress: 47/85
Progress: 48/85
Progress: 49/85
Progress: 50/85
Progress: 51/85
Progress: 52/85
Progress: 53/85
Progress: 54/85
Progress: 55/85
Progress: 56/85
Progress: 57/85
Progress: 58/85
Progress: 59/85
Progress: 60/85
Progress: 61/85
Progress: 62/85
Progress: 63/85
Progress: 64/85
Progress: 65/85
Progress: 66/85
Progress: 67/85
Progress: 68/85
Progress: 69/85
P

In [15]:
# Get the dataframe of all the videos that we will consider

column_names = ['channel_id', 'view_counts', 'uploaded_year', 'category']

df = pd.DataFrame(array_relevant_infos, columns=column_names)

### Remove all the videos that are used for topic modelling

Find the video indices in the dataframe such that:
- more than 100'000 subscribers
- top20 from category/channel/year

In [29]:
df_sub100000 = df[df['channel_id'].isin(set_channelcrawler_100000sub)]
df_top20 = df_sub100000.sort_values(['view_counts'], ascending=False).groupby(['category', 'uploaded_year', 'channel_id']).head(20)
df_top20.head()

,channel_id,view_counts,uploaded_year,category
21326653,UC0C-w0YjGpqDXGB8IHb662A,4468090305,2017,Music
10648298,UCVp3nfGRxmMadNDuVbJSk8A,4295905423,2015,Music
7977749,UCcdwLMPsaU2ezNSJU1nFoBQ,3838039119,2016,Education
4655616,UCmfFGTSsfJVu6CGvL8r75qg,3709532958,2014,Music
13575776,UCN1hnUccO4FD5WfM7ithXaw,3055180938,2015,Music


In [33]:
index_to_remove = set(df_top20.index)

# Process the data

- Transform every video into BoW, according to the topic model vocabulary
- Get the transformed data -> distribution over the topic for each video
- Separate into train set and test set

# Train the classifier

- Train set into train' and validation set, in order to do cross validation
